# **Importamos librerias**

In [9]:
import numpy as np
import pandas as pd
import joblib
import xgboost as xgb



# **Conectamos al Drive**

In [10]:

from google.colab import drive

drive.mount('/content/drive')

dir_data_kaggle='./drive/MyDrive/springleaf/data'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Importamos y transformamos data**

In [11]:


# importamos la lista de funciones de preprocesado

pre_proc_list=joblib.load(dir_data_kaggle+'/pre_pipeline.sav')

scaler=joblib.load(dir_data_kaggle+'/scaler.sav')

In [12]:


# extraemos la data categorica

df_data_cat=pd.read_csv(
                filepath_or_buffer=dir_data_kaggle+'/test.csv'
                    ).drop(['ID'],
                           axis=1).fillna(value=-1).select_dtypes(
                                                            include=['object'])

# aplicamos la transfromacion categorica


for index, transf in enumerate(pre_proc_list):

  df_data_cat[df_data_cat.columns[index]]=transf.transform(
                     list(df_data_cat[df_data_cat.columns[index]].values))


<ipython-input-12-ce6805979c40>:3: DtypeWarning: Columns (8,9,10,11,12,43,157,167,177,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  df_data_cat=pd.read_csv(


In [13]:


# extraemos la data numerica

df_data_num=pd.read_csv(
                filepath_or_buffer=dir_data_kaggle+'/test.csv'
                    ).drop(['ID'],
                           axis=1).fillna(value=-1).select_dtypes(
                                                        include=['int','float'])



# juntamos data numerica y categorica y transformamos

features_data=scaler.transform( pd.concat([df_data_num,df_data_cat],axis=1,
                                 ignore_index=True))


<ipython-input-13-a45419dc70b3>:3: DtypeWarning: Columns (8,9,10,11,12,43,157,167,177,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  df_data_num=pd.read_csv(


# **Importamos el modelo y hacemos predicciones**

In [14]:
# definimos el modelo

model_xgb= xgb.Booster()

model_xgb.load_model(dir_data_kaggle+'/xgb_clf.json')

# definimos la entrada

input_data=xgb.DMatrix(data=features_data)

# realizamos una prediccion

y_pred = model_xgb.predict(input_data)


In [15]:
# limpaimos ram

del features_data, df_data_cat, df_data_num, input_data

In [16]:
# Exportamos el dataframe requerido


df_submit=pd.read_csv(filepath_or_buffer=dir_data_kaggle+'/test.csv',usecols=['ID'])

df_submit['target']=y_pred

print(df_submit)

# guardamos el dataframe

df_submit.to_csv(path_or_buf=dir_data_kaggle+'/submit.csv',index=False)

            ID    target
0            1  0.460275
1            3  0.302153
2            6  0.156475
3            9  0.176637
4           10  0.737986
...        ...       ...
145227  290451  0.002600
145228  290455  0.323424
145229  290456  0.012450
145230  290460  0.200372
145231  290462  0.022595

[145232 rows x 2 columns]
